### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
query='love'
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=f"{url}api-key={nyt_api_key}&q={query}&fq={filter_query}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&sort={sort}"

In [4]:
#Check
page_url=query_url
articles = requests.get(query_url).json()
articles_list = articles["response"]["docs"]
print(json.dumps(articles_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
    # create query with a page number
    # API results show 10 articles at a time

for page in range(0,20):
    page_url=f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        articles = articles["response"]["docs"]
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles:
            reviews_list.append(article)
            content=len(reviews_list)
        # Print the page that was just retrieved
        print(f"the page number retrieved is:{page}") 
        
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"There is an error retrieving articles on page :{page}") 
        break
#print(json.dumps(reviews_list, indent=4))

the page number retrieved is:0
the page number retrieved is:1
the page number retrieved is:2
the page number retrieved is:3
the page number retrieved is:4


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range (0,5):
    print(json.dumps(reviews_list[i], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
' I tried the suggested lambda unction and my process and my procees gave a cleaner result for the title'
'I have inlcuded the lambda function in the comment below in case you would like to check both'

reviews_df = pd.json_normalize(reviews_list)
reviews_df['headline.main2']=reviews_df['headline.main'].str.replace("’s","++s")
reviews_df['title']=reviews_df['headline.main2'].str.extract(r"\u2018(.*?)\u2019", expand=True)
#reviews_df['title']= reviews_df['headline.main2']. apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df['title']=reviews_df['title'].str.replace("++s","'s")
reviews_df['title']=reviews_df['title'].str.rstrip(",")
reviews_df[['headline.main','headline.main2','title']]. head(30)

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}:{item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords2']= reviews_df['keywords'].apply(extract_keywords)
reviews_df.head(10)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_df['title'].tolist()
title_list

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
url2='https://api.themoviedb.org/3/movie/'
tmdb_key_string1 = "&api_key=" + tmdb_api_key
tmdb_key_string2 = "?api_key=" + tmdb_api_key

In [ ]:
  # Make a request for a the full movie details
#movie_url = f"{url2}{movie_id}{tmdb_key_string2}"
  #movie_details = requests.get(movie_url).json()
#print(json.dumps( movie_details, indent=4))

In [ ]:
# Create an empty list to store the results
movie_list=[]
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter=1
# Loop through the titles
for title in title_list:
    request_counter=request_counter+1
    # Check if we need to sleep before making a request
    if request_counter==50:
        time.sleep(1)
        print(" The application is sleeping")
     # Add 1 to the request counter
    else:
        try:
    # Perform a "GET" request for The Movie Database
            query_url= f"{url}{title}{tmdb_key_string1}"
            articles = requests.get(query_url).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
            movie_id = articles['results'][0]['id']

        # Make a request for a the full movie details
            movie_url = f"{url2}{movie_id}{tmdb_key_string2}"

        # Execute "GET" request with url
            movie_details = requests.get(movie_url).json()
        
        # Extract the genre names into a list
            genres=[genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
            lang=[lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list

            prod_country=[country['name'] for country in movie_details['production_countries']]
        # Add the relevant data to a dictionary and
            movie_key_details ={
                            'title': movie_details['title'],
                            'original_title' : movie_details['original_title'],
                            'budget': movie_details['budget'],
                            'original_language': movie_details['original_language'],
                            'homepage':movie_details['homepage'],
                            'overview':movie_details['overview'],
                            'popularity':movie_details['popularity'],
                            'runtime':movie_details['runtime'],
                            'revenue':movie_details['revenue'],
                            'release_date':movie_details['release_date'],
                            'vote_average':movie_details['vote_average'],
                            'vote_count':movie_details['vote_count'],
                            'Genre': genres,
                            'Spoken_Languages': lang,
                            'production_countries': prod_country
                            }
        # append it to the tmdb_movies_list list

            movie_list.append(movie_key_details)
        # Print out the title that was found
            print(f" Found Movie {title}")
        except:
            print(f" Not Found Movie {title} ")

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range (0,5):
    print(json.dumps(movie_list[i], indent=4))

In [ ]:
# Convert the results to a DataFrame
movie_list_df= pd.DataFrame(movie_list)
movie_list_df.head(10)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
